# Biased Clustering

This notebook uses time-biased clustering to detect trends in financial journals.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd drive/MyDrive/ML_Trending_Topics/code

/content/drive/MyDrive/ML_Trending_Topics/code


In [3]:
#import sys
#sys.path.append('/content/drive/MyDrive/Trending-Topics-Dashboard-main-1')

In [4]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from Biased_Clusters import get_clusters_dist, get_clusters_timeline, get_top_keywords, get_silhouette, cal_cluster_bias

In [5]:
# load cleaned data
df = pd.read_csv('../extracted_files/data_cleaned.csv')
df.head()

,Year,Year_Scaled,Year_STD,Month,Keywords,Abstract,Abstract_Cleaned,Abstract Length,Keywords_Cleaned,Number of Keywords,Month_Cleaned
0,2021,2021,566.928668,January,"Motion segmentation,Computer vision,Transmissi...",Many real-world video sequences cannot be conv...,Many real world video sequence cannot convenie...,1594,"['Motion segmentation', 'Computer vision', 'Tr...",11,1
1,2021,2021,566.928668,January,"Generative adversarial networks,Generators,Gal...",Generative adversarial networks (GAN) are trai...,Generative adversarial network GAN trained m...,955,"['Generative adversarial networks', 'Generator...",11,1
2,2021,2021,566.928668,January,"Convolution,Task analysis,Image resolution,Acc...",Many different deep networks have been used to...,Many different deep network used approximate ...,1393,"['Convolution', 'Task analysis', 'Image resolu...",11,1
3,2021,2021,566.928668,January,"Ellipsoids,Shape,Rendering ,computer graphics,...","This paper presents a precise, stable, and inv...",This paper present precise stable invertible...,914,"['Ellipsoids', 'Shape', 'Rendering ', 'compute...",12,1
4,2021,2021,566.928668,January,"Bayes methods,Principal component analysis,Ada...",Robust tensor factorization is a fundamental p...,Robust tensor factorization fundamental proble...,1300,"['Bayes methods', 'Principal component analysi...",11,1


## Biased Clustering

In [6]:
# build trend score table

# load training data
x_vector = np.load('../extracted_files/x_vector.npy', allow_pickle=True)

# steps = [i for i in range(1,1000,50)]
data = []

# get Silhouette score, std year for various bias amount
for m in tqdm(range(1,1000,10)):
    m = m*.01

    try:
        data.append(get_silhouette(df, x_vector, m))
    except ValueError:
        continue

# create a dataframe to store the results
df_result = pd.DataFrame(data)

# save df_result to csv file
df_result.to_csv('table_trend_score.csv', index=False)

# display the first 5 rows of df_result
df_result.head()

  0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
  1%|          | 1/100 [00:04<07:43,  4.68s/it]/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
  2%|▏         | 2/100 [00:07<05:42,  3.50s/it]/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
  3%|▎         | 3/100 [00:10<05:15,  3.25s/it]/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will cha

,step,Silhouette Score,silhouette_clsuter_1,std_year_cluster_1,silhouette_by_std_year_cluster_1,silhouette_clsuter_2,std_year_cluster_2,silhouette_by_std_year_cluster_2,silhouette_clsuter_3,std_year_cluster_3,...,silhouette_clsuter_14,std_year_cluster_14,silhouette_by_std_year_cluster_14,silhouette_clsuter_15,std_year_cluster_15,silhouette_by_std_year_cluster_15,std_tfidf,avg_std,avg_std_year,avg_silhouette_by_std_year
0,0.01,0.209274,0.266208,3.587119,0.074212,0.182352,3.470745,0.052540,0.125995,3.474238,...,0.264512,3.423189,0.077271,0.121207,3.722586,0.032560,1.744931,0.118192,3.470001,0.064044
1,0.11,0.213618,0.261353,3.531339,0.074010,0.416470,2.571515,0.161955,0.094641,3.215563,...,0.181855,3.727567,0.048787,0.175641,3.655019,0.048055,1.741179,0.114159,3.458951,0.065224
2,0.21,0.213832,0.200587,3.312210,0.060560,0.151807,3.352106,0.045287,0.426990,3.666850,...,0.294290,3.505192,0.083958,0.415735,2.480856,0.167577,1.736381,0.113608,3.439951,0.066536
3,0.31,0.188113,0.453748,3.560798,0.127429,0.135004,3.192782,0.042284,0.191647,3.576943,...,0.185091,3.565415,0.051913,0.121187,3.107739,0.038995,1.740215,0.144114,3.026803,0.063613
4,0.41,0.140208,-0.046834,1.981494,-0.023636,0.471232,3.407901,0.138276,-0.089879,1.947047,...,0.126154,2.395083,0.052672,0.456878,3.359413,0.135999,1.745372,0.178397,2.453138,0.056101


In [7]:
len(df)

4604

In [8]:
import json
import scipy.sparse

# load terms sparse matrix
terms_sparse_matrix = scipy.sparse.load_npz('../extracted_files/terms_sparse_matrix.npz')

# load terms label
with open("../extracted_files/terms_label.txt", "r") as fp:
    terms_label = json.load(fp)

terms_sparse_matrix = pd.DataFrame(terms_sparse_matrix.toarray())
terms_sparse_matrix = terms_sparse_matrix.iloc[:len(x_vector), :]

# make predictions
summary, predictions = cal_cluster_bias(df, x_vector, terms_sparse_matrix, terms_label, 0.57)


/content/drive/MyDrive/ML_Trending_Topics/code/Biased_Clusters.py:184: RuntimeWarning: divide by zero encountered in scalar divide
  trend_score.append(np.mean(tmp_silhotte / bias_avg_std_year[-1]))


In [9]:
summary

,Topic Id,Terms,Timeline,Number of Articles,Article %,Trend Score,Silhouette Score,Bias Avg Std Year
0,1,"computer, networks, algorithms, extraction, sh...",2010-2023,3488,75.760209,-0.027090,-0.097037,3.582030
1,2,"imagingdiffractive, masks, two, masksphase, di...",2020,2,0.043440,inf,1.000000,0.000000
2,3,"measurement, predictive, visualization, models...",2021,2,0.043440,inf,1.000000,0.000000
3,4,"learningtrainingobject, detectionbenchmarking,...",2019,2,0.043440,inf,0.999999,0.000000
4,5,"databases, structure, assessmentstructure, vis...",2022,2,0.043440,inf,1.000000,0.000000
5,6,"modelinggravityvisual, brightness, movements, ...",2020,2,0.043440,inf,1.000000,0.000000
6,7,"heterogeneous, transfer, feature, measurement,...",2019,2,0.043440,inf,1.000000,0.000000
7,8,"flowmotion, flowlagrangian, tomographyfeature,...",2015,2,0.043440,inf,1.000000,0.000000
8,9,"modelsrandom, random, mixedeffects, researchlo...",2018,2,0.043440,inf,1.000000,0.000000
9,10,"face, dense, markov, human, priorslocallyconsi...",2015-2016,4,0.086881,1.100798,0.550399,0.500000


In [10]:
# #Code to extract all the frequently occuring terms in the reseach papers
# words = summary["Terms"].str.split(", ").explode().unique()

# # save the list of words to a text file
# pd.Series(words).to_csv("extracted_files/frequent_terms.csv", index=False)
# # len(words)
# # words

In [11]:
predictions.head()

,Year,Year_Scaled,Year_STD,Month,Keywords,Abstract,Abstract_Cleaned,Abstract Length,Keywords_Cleaned,Number of Keywords,Month_Cleaned,Topic Id
0,2021,2021,566.928668,January,"Motion segmentation,Computer vision,Transmissi...",Many real-world video sequences cannot be conv...,Many real world video sequence cannot convenie...,1594,"['Motion segmentation', 'Computer vision', 'Tr...",11,1,1
1,2021,2021,566.928668,January,"Generative adversarial networks,Generators,Gal...",Generative adversarial networks (GAN) are trai...,Generative adversarial network GAN trained m...,955,"['Generative adversarial networks', 'Generator...",11,1,1
2,2021,2021,566.928668,January,"Convolution,Task analysis,Image resolution,Acc...",Many different deep networks have been used to...,Many different deep network used approximate ...,1393,"['Convolution', 'Task analysis', 'Image resolu...",11,1,1
3,2021,2021,566.928668,January,"Ellipsoids,Shape,Rendering ,computer graphics,...","This paper presents a precise, stable, and inv...",This paper present precise stable invertible...,914,"['Ellipsoids', 'Shape', 'Rendering ', 'compute...",12,1,15
4,2021,2021,566.928668,January,"Bayes methods,Principal component analysis,Ada...",Robust tensor factorization is a fundamental p...,Robust tensor factorization fundamental proble...,1300,"['Bayes methods', 'Principal component analysi...",11,1,1


In [12]:
get_clusters_dist(predictions).head()

,Year,Topic Id,Number of Articles
0,2010,1,138
1,2010,12,7
2,2010,13,13
3,2010,14,1
4,2010,15,19


In [13]:
df_temp = get_clusters_dist(predictions).sort_values(['Year', 'Number of Articles'], ascending=[True, False])
df_temp

,Year,Topic Id,Number of Articles
0,2010,1,138
4,2010,15,19
2,2010,13,13
1,2010,12,7
3,2010,14,1
...,...,...,...
78,2023,1,68
81,2023,14,11
80,2023,12,5
82,2023,15,5


In [14]:
# filtered_df = df_temp.groupby('Year')['Number of Articles'].idxmax()
# df_trending_id = df_temp.loc[filtered_df]

year_counts = df_temp.groupby('Year')['Number of Articles'].count().sort_values(ascending=False)
year_order = year_counts.index.tolist()

# create a dictionary to map year to rank
year_rank = {}
for i, year in enumerate(year_order):
    year_rank[year] = i

df_temp['YearRank'] = df_temp['Year'].map(year_rank)
df_temp.sort_values(by=['YearRank', 'Number of Articles'], ascending=[True, False], inplace=True)

# group by year and topic, and get the topic with the highest count for each year
grouped = df_temp.groupby(['Year', 'Topic Id']).agg({'Number of Articles': 'sum'})
grouped.reset_index(inplace=True)
filtered_df = grouped.groupby('Year')['Number of Articles'].idxmax()
df_trending_id = grouped.loc[filtered_df]
df_trending_id.sort_values(by='Year', inplace=True)

In [15]:
df_trending_id

,Year,Topic Id,Number of Articles
0,2010,1,138
5,2011,1,150
9,2012,1,166
14,2013,1,170
19,2014,1,148
24,2015,1,302
31,2016,1,295
37,2017,1,268
42,2018,1,334
48,2019,1,322


In [16]:
df_trending_topics = pd.merge(df_trending_id, predictions[['Topic Id', 'Keywords']], on='Topic Id')
df_trending_topics

,Year,Topic Id,Number of Articles,Keywords
0,2010,1,138,"Motion segmentation,Computer vision,Transmissi..."
1,2010,1,138,"Generative adversarial networks,Generators,Gal..."
2,2010,1,138,"Convolution,Task analysis,Image resolution,Acc..."
3,2010,1,138,"Bayes methods,Principal component analysis,Ada..."
4,2010,1,138,"Entropy,Kernel,Estimation,Probability distribu..."
...,...,...,...,...
48827,2023,1,68,Feature extraction;Big data;Algorithm design a...
48828,2023,1,68,Markov processes;Stochastic processes;Gaussian...
48829,2023,1,68,Image sequences;Cameras;Gait recognition;Visua...
48830,2023,1,68,Chaos theory;Hidden Markov models;Analytical m...


In [17]:
# group the dataframe by year
grouped = df_trending_topics.groupby('Year')

# define a function to get the most recurring keywords
def get_most_common_keywords(group):
    # join all keywords into a single string
    keywords_str = ', '.join(group['Keywords'])
    
    # split the string into individual keywords if it is not empty
    if keywords_str:
        keywords = keywords_str.split(', ')
    else:
        keywords = []
    
    # count the frequency of each keyword
    keyword_counts = pd.Series(keywords).value_counts()
    
    # get the first 10 keywords that occur the most
    most_common_keywords = keyword_counts.head(11).index.tolist()
    most_common_keywords = [kw.strip() for kw in most_common_keywords if kw.strip()]

    # create a new dataframe with the most common keywords and other columns from the original dataframe
    new_row = pd.DataFrame({
        'Year': [group['Year'].iloc[0]],
        'Topic Id': [group['Topic Id'].iloc[0]],
        'Number of Articles': [group['Number of Articles'].iloc[0]],
        'Keywords': [', '.join(most_common_keywords)]
    })
    return new_row

# apply the function to each group and concatenate the results into a new dataframe
trending_topics = pd.concat([get_most_common_keywords(group) for name, group in grouped])

# reset the index of the new dataframe
trending_topics = trending_topics.reset_index(drop=True)

# print the new dataframe
print(trending_topics)

    Year  Topic Id  Number of Articles  \
0   2010         1                 138   
1   2011         1                 150   
2   2012         1                 166   
3   2013         1                 170   
4   2014         1                 148   
5   2015         1                 302   
6   2016         1                 295   
7   2017         1                 268   
8   2018         1                 334   
9   2019         1                 322   
10  2020         1                 333   
11  2021         1                 399   
12  2022         1                 395   
13  2023         1                  68   

                                             Keywords  
0   Artificial intelligence, Training, Task analys...  
1   Artificial intelligence, Training, Task analys...  
2   Artificial intelligence, Training, Task analys...  
3   Artificial intelligence, Training, Task analys...  
4   Artificial intelligence, Training, Task analys...  
5   Artificial intelligence, Trai

In [18]:
keywords = trending_topics[trending_topics['Year'] == 2015]['Keywords']
for keyword in keywords:
    print(keyword)

Artificial intelligence, Training, Task analysis, Data models, Feature extraction, Markov random fields, Servers, Computer vision, Conferences, Internet of Things


In [19]:
#Finding the second highest trending topics
year_counts = df_temp.groupby('Year')['Number of Articles'].count().sort_values(ascending=False)
year_order = year_counts.index.tolist()

# create a dictionary to map year to rank
year_rank = {}
for i, year in enumerate(year_order):
    year_rank[year] = i

df_temp['YearRank'] = df_temp['Year'].map(year_rank)
df_temp.sort_values(by=['YearRank', 'Number of Articles'], ascending=[True, False], inplace=True)

# group by year and topic, and get the topic with the second-highest count for each year
grouped = df_temp.groupby(['Year', 'Topic Id']).agg({'Number of Articles': 'sum'})
grouped.reset_index(inplace=True)
grouped['Rank'] = grouped.groupby('Year')['Number of Articles'].rank(method='dense', ascending=False)
filtered_df = grouped.loc[grouped['Rank'] == 2].groupby('Year')['Number of Articles'].idxmax()
df_second_trending_id = grouped.loc[filtered_df]
df_second_trending_id.sort_values(by='Year', inplace=True)


In [20]:
df_second_trending_topics = pd.merge(df_second_trending_id[['Year',	'Topic Id',	'Number of Articles']], predictions[['Topic Id', 'Keywords']], on='Topic Id')
df_second_trending_topics

,Year,Topic Id,Number of Articles,Keywords
0,2010,15,19,"Ellipsoids,Shape,Rendering ,computer graphics,..."
1,2010,15,19,"Two dimensional displays,Pose estimation,Detec..."
2,2010,15,19,"Image restoration,Degradation,Rain,Deconvoluti..."
3,2010,15,19,"Cameras,Calibration,Three-dimensional displays..."
4,2010,15,19,"Vegetation,Estimation,Forestry,Nonhomogeneous ..."
...,...,...,...,...
4019,2023,14,11,Stochastic processes;Motion pictures;Pattern m...
4020,2023,14,11,Correlation;Cameras;Image coding;Training;Thre...
4021,2023,14,11,Visualization;Computational modeling;Data mode...
4022,2023,14,11,Sensors;Head;Magnetic heads;Surveillance;Camer...


In [21]:
# group the dataframe by year
grouped = df_second_trending_topics.groupby('Year')

# define a function to get the most recurring keywords
def get_most_common_keywords(group):
    # join all keywords into a single string
    keywords_str = ', '.join(group['Keywords'])
    
    # split the string into individual keywords if it is not empty
    if keywords_str:
        keywords = keywords_str.split(', ')
    else:
        keywords = []
    
    # count the frequency of each keyword
    keyword_counts = pd.Series(keywords).value_counts()
    
    # get the first 10 keywords that occur the most
    most_common_keywords = keyword_counts.head(11).index.tolist()
    most_common_keywords = [kw.strip() for kw in most_common_keywords if kw.strip()]

    # create a new dataframe with the most common keywords and other columns from the original dataframe
    new_row = pd.DataFrame({
        'Year': [group['Year'].iloc[0]],
        'Topic Id': [group['Topic Id'].iloc[0]],
        'Number of Articles': [group['Number of Articles'].iloc[0]],
        'Keywords': [', '.join(most_common_keywords)]
    })
    return new_row

# apply the function to each group and concatenate the results into a new dataframe
second_trending_topics = pd.concat([get_most_common_keywords(group) for name, group in grouped])

# reset the index of the new dataframe
second_trending_topics = second_trending_topics.reset_index(drop=True)

# print the new dataframe
print(second_trending_topics)

    Year  Topic Id  Number of Articles  \
0   2010        15                  19   
1   2011        13                  18   
2   2012        15                  15   
3   2013        15                  23   
4   2014        13                  17   
5   2015        15                  33   
6   2016        15                  40   
7   2017        13                  38   
8   2018        15                  30   
9   2019        13                  30   
10  2020        14                  40   
11  2021        14                  57   
12  2022        14                  53   
13  2023        14                  11   

                                             Keywords  
0   speech recognition, speaker recognition, LDA, ...  
1   structure from motion, 3D reconstruction, SLAM...  
2   speech recognition, speaker recognition, LDA, ...  
3   speech recognition, speaker recognition, LDA, ...  
4   structure from motion, 3D reconstruction, SLAM...  
5   speech recognition, speaker r

In [22]:
keywords = second_trending_topics[second_trending_topics['Year'] == 2020]['Keywords']
for keyword in keywords:
    print(keyword)

Computational modeling, Computer vision, 5G mobile communication, Deep learning, deep learning, image object recognition, non-iterative learning, Performance evaluation, Industrial Internet of Things, Image coding;Video compression;Optical imaging;Motion estimation;Optical distortion;Estimation;Adaptive optics;Video compression;neural network;end-to-end optimization;image compression


In [23]:
# save prediction
summary.to_csv('summary_bias_0.57.csv', index=False)
predictions.to_csv('predictions_0.57.csv', index=False)
trending_topics.to_csv('trending_topics_0.57.csv',index=False)
second_trending_topics.to_csv('second_trending_topic_0.57.csv',index=False)